In [1]:
import numpy as np
import pandas as pd

from scipy.stats import norm
from matplotlib import pyplot as plt
from src.data_handling import load_csv_data
from src.vol_forecasting import GARCHVolForecast, StochasticVolForecast
from arch import arch_model

def stochastic_volatility(data, n_days, n_sim = 1000):
    """
    Translating StochasticVolForecast class into a single function
    """
    stochVolForecast = StochasticVolForecast(data)
    stochVolForecast.fit_model()
    vol = stochVolForecast.forecast_volatility(n_days, n_sim)


    return vol

def garch_volatility(data, n_days):
    """
    Translating GARCHVolForecast class into a single function
    """
    GARCH_model = GARCHVolForecast(data)
    GARCH_model.fit_model()
    vol = GARCH_model.forecast_volatility(n_days)

    return vol

In [2]:
DATA_PATH = 'DATA/processed/df.csv'
df = load_csv_data(DATA_PATH)
df["IRX_Close"] = df["IRX_Close"].apply(abs)
df.head()

,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Adj Close,AAPL_Volume,AMZN_Open,AMZN_High,AMZN_Low,AMZN_Close,...,VIX_Open,VIX_High,VIX_Low,VIX_Close,VIX_Adj Close,IRX_Open,IRX_High,IRX_Low,IRX_Close,IRX_Adj Close
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,27.847500,27.860001,26.837500,27.332500,24.531765,212818400.0,15.6290,15.7375,15.3480,15.4260,...,17.760000,20.139999,17.049999,17.790001,17.790001,0.030,0.030,0.015,0.015,0.015
2015-01-05,27.072500,27.162500,26.352501,26.562500,23.840672,257142000.0,15.3505,15.4190,15.0425,15.1095,...,19.190001,21.290001,19.190001,19.920000,19.920000,0.008,0.018,0.003,0.003,0.003
2015-01-06,26.635000,26.857500,26.157499,26.565001,23.842909,263188400.0,15.1120,15.1500,14.6190,14.7645,...,20.330000,22.900000,19.520000,21.120001,21.120001,0.023,0.023,0.018,0.020,0.020
2015-01-07,26.799999,27.049999,26.674999,26.937500,24.177242,160423600.0,14.8750,15.0640,14.7665,14.9210,...,20.150000,20.719999,19.040001,19.309999,19.309999,0.023,0.025,0.020,0.020,0.020
2015-01-08,27.307501,28.037500,27.174999,27.972500,25.106180,237458000.0,15.0160,15.1570,14.8055,15.0230,...,17.930000,18.090000,16.990000,17.010000,17.010000,0.020,0.020,0.015,0.018,0.018


In [54]:
N_DAYS = 5
VOL_LOOKBACK = 10
CONFIDENCE = 1.96

def garch_error(n_days, vol_lookback):
    # Compute date_start directly using vectorized operations
    date_idx = np.arange(5, len(df) - n_days, 5)
    date_start = np.clip(date_idx - vol_lookback, 0, None)

    # Use a list comprehension for forecasted_vol, but avoid the range(len(date_idx))
    forecasted_vol = [garch_volatility(df.iloc[start:end], n_days) for start, end in zip(date_start, date_idx)]

    spx_close = df["SPX_Close"].values

    # Compute the differences using vectorized operations
    end_values = spx_close[date_idx + n_days]
    start_values = spx_close[date_idx]
    difference = np.abs(end_values - start_values) - CONFIDENCE * np.array(forecasted_vol)
    return sum(difference)

# [i*N_DAYS for i in range(1, int(len(df)/n_days) - n_days)]

In [51]:
garch_error(N_DAYS, VOL_LOOKBACK)

-10970.877867407098

In [57]:
from scipy.optimize import minimize

# Define the objective function
def objective(params):
    n_days, vol_lookback = params
    return garch_error(n_days, vol_lookback)

n_days_param = np.arange(5, 31, 2)
vol_lookback_param = np.arange(5, 100, 5)

import itertools

params = list(itertools.product(n_days_param, vol_lookback_param))

local_min = 0
param_min = None
for param in params:
    print(param)
    res = objective(param)
    if res< local_min:
        local_min = res
        param_min = param

(5, 5)
(5, 10)
(5, 15)
(5, 20)
(5, 25)
(5, 30)
(5, 35)
(5, 40)
(5, 45)
(5, 50)
(5, 55)
(5, 60)
(5, 65)
(5, 70)
(5, 75)
(5, 80)
(5, 85)
(5, 90)
(5, 95)
(7, 5)
(7, 10)
(7, 15)
(7, 20)
(7, 25)
(7, 30)
(7, 35)
(7, 40)
(7, 45)
(7, 50)
(7, 55)
(7, 60)
(7, 65)
(7, 70)
(7, 75)
(7, 80)
(7, 85)
(7, 90)
(7, 95)
(9, 5)
(9, 10)
(9, 15)
(9, 20)
(9, 25)
(9, 30)
(9, 35)
(9, 40)
(9, 45)
(9, 50)
(9, 55)
(9, 60)
(9, 65)
(9, 70)
(9, 75)
(9, 80)
(9, 85)
(9, 90)
(9, 95)
(11, 5)
(11, 10)
(11, 15)
(11, 20)
(11, 25)
(11, 30)
(11, 35)
(11, 40)
(11, 45)
(11, 50)
(11, 55)
(11, 60)
(11, 65)
(11, 70)
(11, 75)
(11, 80)
(11, 85)
(11, 90)
(11, 95)
(13, 5)
(13, 10)
(13, 15)
(13, 20)
(13, 25)
(13, 30)
(13, 35)
(13, 40)
(13, 45)
(13, 50)
(13, 55)
(13, 60)
(13, 65)
(13, 70)
(13, 75)
(13, 80)
(13, 85)
(13, 90)
(13, 95)
(15, 5)
(15, 10)
(15, 15)
(15, 20)
(15, 25)
(15, 30)
(15, 35)
(15, 40)
(15, 45)
(15, 50)
(15, 55)
(15, 60)
(15, 65)
(15, 70)
(15, 75)
(15, 80)
(15, 85)
(15, 90)
(15, 95)
(17, 5)
(17, 10)
(17, 15)
(17, 20)
(1

In [58]:
local_min

-91906.29048272167

In [59]:
param_min

(5, 95)